In [1]:
import numpy as np
import copy
from collections import OrderedDict

from src.supplementary import Define_Derivatives
from src.term import Term
from src.trainer import Equation_Trainer

from src.evo_optimizer import Operator_director, Operator_builder
from src.token_family import Evaluator, Token_family

В этом примере - пробуем восстановить ОДУ вида $u \sin x + u' \cos x = 1$, которое имеет решение $u = \sin x + C \cos x$. <br> 
Загружаем уже подготовленные данные данные и формируем из них np.ndarray для оценки значений исходной функции и её производных (в это случае - одной производной):

In [2]:
u_initial = np.load('Preprocessing/Fill366/fill366.npy')
print(u_initial.shape)

derivatives = np.load('Preprocessing/Fill366/Derivatives.npy')
variables = np.ones((1 + derivatives.shape[1], ) + u_initial.shape)
variables[0, :] = u_initial
for i_outer in range(0, derivatives.shape[1]):
    variables[i_outer+1] = derivatives[:, i_outer].reshape(variables[i_outer+1].shape) 

(1000,)


Начинаем задавать множества "токенов", из которых будет строиться искомое уравнение. Для простоты и используя знание, что уравнение состоит из тригонометрических ф-ций и производных, зададим их через семейство множеств, где первое мн-во - производные и исходная функция, а второе - тригонометрические функции. Сначала, зададим функцию, которые позволяют оценить значений токенов из множества с производными в исследуемой области и вернуть векторы их значений:

In [3]:
def simple_function_evaluator(token, t_params, eval_params):
    
    '''
    
    Example of the evaluator of token values, appropriate for case of trigonometric functions to be calculated on grid, with results in forms of tensors
    
    Parameters
    ----------
    token: {'u', 'du/dx', ...}
        symbolic form of the function to be evaluated: 
    token_params: dictionary: key - symbolic form of the parameter, value - parameter value
        names and values of the parameters for trigonometric functions: amplitude, frequency & dimension
    eval_params : dict
        Dictionary, containing parameters of the evaluator: in this example, it contains coordinates np.ndarray with pre-calculated values of functions, 
        names of the token parameters (power). Additionally, the names of the token parameters must be included with specific key 'params_names', 
        and parameters range, for which each of the tokens if consedered as "equal" to another, like sin(1.0001 x) can be assumed as equal to (0.9999 x)
    
    Returns
    ----------
    value : numpy.ndarray
        Vector of the evaluation of the token values, that shall be used as target, or feature during the LASSO regression.
        
    '''
    
    
    assert 'token_matrices' in eval_params
    value = copy.deepcopy(eval_params['token_matrices'][token])
    value = value**(t_params['power'])
    return value    

In [20]:
token_names_der = list(Define_Derivatives(u_initial.ndim, max_order = 1))
simple_functions = {}
for var_idx in np.arange(variables.shape[0]):
    simple_functions[token_names_der[var_idx]] = variables[var_idx]    

derivatives_tokens = Token_family('Derivatives') # Задаём объект системы множеств
# Задаём параметры оценщика значений "токена":
der_eval_params = {'token_matrices':simple_functions, 'params_names':['power'], 'params_equality':{'power' : 0}}
# Настраиваем оценщик значений "токенов" через ф-цию simple_function_evaluator и соотв. параметры 
derivatives_tokens.set_evaluator(simple_function_evaluator, **der_eval_params) 
# Для токенов из производных единственный параметр - степень, в которой они пристутсвуют в слагаемом. 
# В этом эксперименте, чтобы избежать некоторых тождеств, связанных с решением исходного уравнения, ограничися 1ой степенью 
der_token_params = OrderedDict([('power', (1, 1))])
# Задаём названия токенов и параметры:
derivatives_tokens.set_params(token_names_der, der_token_params)
# Задаём статус токенов: хотя бы один из них обязательно использовать в слагаемом, и те, которые есть в правой части,
# не могут быть использованы в левой: 
derivatives_tokens.set_status(unique_specific_token=True, mandatory = True, unique_for_right_part = True)

Test evaluation performed correctly


Проведём аналогичные операции с тригонометрическими ф-циями:

In [5]:
def trigonometric_evaluator(token, token_params, eval_params):
    
    '''
    
    Example of the evaluator of token values, appropriate for case of trigonometric functions to be calculated on grid, with results in forms of tensors
    
    Parameters
    ----------
    token: {'sin', 'cos'}
        symbolic form of the function to be evaluated: 
    token_params: dictionary: key - symbolic form of the parameter, value - parameter value
        names and values of the parameters for trigonometric functions: amplitude, frequency & dimension
    eval_params : dict
        Dictionary, containing parameters of the evaluator: in this example, it contains coordinates np.meshgrid with coordinates for points, 
        names of the token parameters (frequency, axis and power). Additionally, the names of the token parameters must be included with specific key 'params_names', 
        and parameters range, for which each of the tokens if consedered as "equal" to another, like sin(1.0001 x) can be assumed as equal to (0.9999 x)
    
    Returns
    ----------
    value : numpy.ndarray
        Vector of the evaluation of the token values, that shall be used as target, or feature during the LASSO regression.
        
    '''
    
    assert 'grid' in eval_params
    trig_functions = {'sin' : np.sin, 'cos' : np.cos}
    function = trig_functions[token]
    grid_function = np.vectorize(lambda *args: function(token_params['freq']*args[token_params['dim']])**token_params['power'])
    value = grid_function(*eval_params['grid'])
    return value

Зададим вспомогательную функцию set_grid, возвращающую равномерную сетку, соответствующую входым данным (которые с производными) на основе их параметров их матриц и заданных шагов:

In [6]:
def set_grid(template_matrix, steps):
    assert np.ndim(template_matrix) == len(steps)
    sd_grids = []
    for dim in np.arange(np.ndim(template_matrix)):
        sd_grids.append(np.arange(0, template_matrix.shape[dim]*steps[dim], steps[dim]))
    return np.meshgrid(*sd_grids, indexing = 'ij')

In [8]:
token_names_trig = ['sin', 'cos']
step = 4*np.pi/999
grid = set_grid(variables[0], steps = (step,))

trigonometric_tokens = Token_family('Trigonometric')
trig_eval_params = {'grid':grid, 'params_names':['power',  'freq', 'dim'], 'params_equality':{'power': 0, 'freq':0.05, 'dim':0}}
trigonometric_tokens.set_evaluator(trigonometric_evaluator, **trig_eval_params)
trig_token_params = OrderedDict([('power', (1, 1)), ('freq', (0.9, 1.1)), ('dim', (0, u_initial.ndim))])
trigonometric_tokens.set_params(token_names_trig, trig_token_params)
trigonometric_tokens.set_status(unique_token_type=True)

Test evaluation performed correctly


Из множеств токенов зададим систему, представленную как лист:

In [9]:
tokens = [trigonometric_tokens, derivatives_tokens]

Для задания эволюционного оператора воспользуемся стандартным подходом (есть все виды мутации и кроссовера, описанные в статье, для поиска коэффициентов уравнения (действительных чисел; тригонометрическая часть коэф-та ищется в эволюционном алгоритме) используем LASSO-регрессию & оцениваем фитнесс-функцию как невязку левой и правой части уравнений), реализованном в директоре, руководящем строителем эволюционных операторов:

In [10]:
director = Operator_director()
director.operator_assembly(sparcity = 0.5)   

Задаём тренер уравнений, в который посылаем систему токенов, а также задаём параметры популяции (размер), длину уравнений и макс. число токенов-множителей в каждом слагаемом:

In [21]:
Trainer = Equation_Trainer(tokens = tokens, basic_terms = [])
Trainer.parameters_grid(('pop_size', 'eq_len', 'max_factors', 'test_output'), 
                        (20, 4, 2, False))

Проводим обучение:

In [25]:
Trainer.train(epochs = 150, evolutionary_operator = director.constructor.operator)    

Using parameters from grid
0 0.06135808303707546
<class 'list'>
1 0.06135808303707546
<class 'list'>
2 0.06135808303707546
<class 'list'>
3 0.06135808303707546
<class 'list'>
4 0.06135808303707546
<class 'list'>
5 0.06251574941816525
<class 'list'>
6 0.06251574941816525
<class 'list'>
7 0.06251574941816525
<class 'list'>
8 0.06251574941816525
<class 'list'>
9 0.06251574941816525
<class 'list'>
10 0.06251574941816525
<class 'list'>
11 0.06251574941816525
<class 'list'>
12 0.06251574941816525
<class 'list'>
13 0.06251574941816525
<class 'list'>
14 0.06251574941816526
<class 'list'>
15 0.06251574941816526
<class 'list'>
16 0.06251574941816526
<class 'list'>
17 0.06251574941816526
<class 'list'>
18 0.06271189855142491
<class 'list'>
19 0.06271189855142491
<class 'list'>
20 0.06271189855142491
<class 'list'>
21 0.06271189855142491
<class 'list'>
22 0.06271189855142491
<class 'list'>
23 0.06271189855142491
<class 'list'>
24 0.06271189855142491
<class 'list'>
25 0.06271189855142491
<class 'li

В этом эксперименте может получиться не только искомое уравнение, но и его аналоги, решениями которых также является функция $u$